In [ ]:
pip install yargy

In [ ]:
pip install ipymarkup

In [3]:
text1 = '''Изобретение относится к сельскому хозяйству и предназначено для повышения урожайности и защиты растений от комплекса фитопатогенов.
Известен способ защиты растений от фитопатогенов и стрессов, а также повышения продуктивности сельскохозяйственных культур при использовании препарата Альбит 
(Злотников А.К., Алехин В.Т. и др. Биопрепарат альбит для повышения урожая и защиты растений: опыты, рекомендации, результаты применения. - М., Агрорус, 2008, 248 с.). 
Он создан на основе продукта метаболизма бактерий - поли-бета-гидроксимасляной кислоты. 
Однако технология наработки препарата достаточно сложна и требует дорогостоящего оборудования.
Известен способ повышения эффективности защиты от фитопатогенов увеличения урожайности сельскохозяйственных культур при использовании биостимулятора на основе вермикомпоста - 
вермистим.'''

In [ ]:
import pymorphy2
import re
['относится', 'предназначено', 'для', 'при', 'для', 'создан', 'поли', 'требует', 'при', 'вермистим', 'вермистим', 'вермистим', 'вермистим', 'вермистим', 'вермистим']
words=re.findall(r"[\w']+", text1)
morph = pymorphy2.MorphAnalyzer()
actionVerbs = []
tagCount = 0

for word in words:

  for opencorporaTag in morph.parse(word):
    # print(opencorporaTag.normalized)
    if((opencorporaTag.tag.POS == 'VERB' and opencorporaTag.normalized.tag.POS == 'INFN') and opencorporaTag.tag.POS != 'PREP'):
      print(opencorporaTag)
      actionVerbs.append(opencorporaTag.normal_form)
    tagCount += 1
  tagCount = 0


# print(morph.parse("уменьшение"))
t = morph.parse(words[1])[0].normalized.tag
# if('INFN' in t):
  # print("Verb")
actionVerbs = list(set(actionVerbs))

print(actionVerbs)

# and 'PREP' in opencorporaTag.tag == False

In [ ]:
import spacy
import os
from spacy.lang.ru.examples import sentences

cwd = os.getcwd()
print(cwd)

nlp = spacy.load("ru_core_news_md")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

In [5]:
actionVerbs = [
    'уменьшение',
    'повышение',
    'увеличение',
    'улучшение',
    'разделение',
    'расширение',
    'обеспечение',
    'компенсация',
    'выполнение',
    'облучение',
    'реализация',
    'предотвращение',
    'снижение',
    'исключение',
    'ускорение',
    'создание',
    'препятствие',
    'корректировать',
    'стабилизация',
    'формирование',
    'устранение',
    'посадку',
    'присыпать',
    'техническая задача',
    'оборачивание',
    'защита'
]

In [ ]:
with open("Отглагольные существительные.txt") as file:
    lines = [line.rstrip() for line in file]
lines

In [ ]:
from yargy.tokenizer import MorphTokenizer

TOKENIZER = MorphTokenizer()
list(TOKENIZER("повышения "))

In [7]:
from ipymarkup import show_span_ascii_markup as show_markup

import json
import pymorphy2

from yargy import rule, Parser, or_, and_
from yargy.predicates import eq, type, gram, normalized
from yargy.pipelines import morph_pipeline

# Создание правил для "любого" слова
WORD = or_(
    gram('NOUN'),
    gram('ADJF'),
    gram('VERB')
)

# Создание правил для сущности объект
OBJ1 = rule(
    and_(
      gram('NOUN'),
      gram('gent')  
    ) 
)
OBJ2 = rule(
    OBJ1.optional(),
    gram('ADJF').optional(),
    OBJ1,
    WORD
)
OBJ3 = rule(
    or_(
        rule(
            or_(OBJ1,OBJ2).repeatable(),
            rule(eq('и')),
            or_(OBJ1,OBJ2).repeatable(),
        ),
        rule(
            or_(OBJ1,OBJ2).repeatable(),
            rule(eq('и')),
            or_(OBJ1,OBJ2).repeatable(),
        )
    )
)

# Создание правил для сущности условие
CON1 = rule(
    and_(
        WORD,
        or_(
          gram('loct'),
          gram('ablt')
        )
    )
)
CON2 = rule(
    CON1,
    rule(
        and_(
        WORD,
        gram('gent')
      )
    ).repeatable()
)
CON3 = rule(
    or_(
        rule(
            CON1,
            rule(eq('и')),
            and_(
                WORD,
                gram('nomn')
            )
        ),
        rule(
            CON2,
            rule(eq('и')),
            and_(
                WORD,
                gram('nomn')
            )
        )
    )
)

# Правила струстуры AOC
ACTION = rule(morph_pipeline(actionVerbs))
OBJECT = rule(or_(
    OBJ1,
    OBJ2,
    OBJ3
))
CONDITION = rule(or_(
    CON1,
    CON2,
    CON3
).optional())

# Интерпретация и визуализация результатов
AOC = rule(ACTION, OBJECT, CONDITION)
parser = Parser(AOC)
matches = list(parser.findall(text1))
spans = [_.span for _ in matches]
show_markup(text1, spans)

result = []
sentens = ""
counter = 0
morph = pymorphy2.MorphAnalyzer()
# show_markup(text1, spans)
for matche in matches:
  for i in matche:
    for t in i:
      if(isinstance(t, int) == False):
        if (counter == 0):
          res = morph.parse(t.value)[0].normal_form
          sentens += res + " "
        if(counter != 0):
          sentens += t.value + " "
        counter += 1
      else:
        if(sentens != ""):
          sentens = sentens[:-1]
          result.append(sentens)
          sentens = ""
          counter = 0

result


Изобретение относится к сельскому хозяйству и предназначено для 
повышения урожайности и защиты растений от комплекса фитопатогенов.
───────────────────────────────────────                            
Известен способ защиты растений от фитопатогенов и стрессов, а также 
                ───────────────                                      
повышения продуктивности сельскохозяйственных культур при 
──────────────────────────────────────────────────────────
использовании препарата Альбит 
────────────────────────────── 
(Злотников А.К., Алехин В.Т. и др. Биопрепарат альбит для повышения 
                                                          ──────────
урожая и защиты растений: опыты, рекомендации, результаты применения. 
────────────────────────                                              
- М., Агрорус, 2008, 248 с.). 
Он создан на основе продукта метаболизма бактерий - поли-бета-
гидроксимасляной кислоты. 
Однако технология наработки препарата достаточно сложна и требует 
дорогосто

['повышение урожайности и защиты растений',
 'защита растений',
 'повышение продуктивности сельскохозяйственных культур при использовании препарата Альбит',
 'повышение урожая и защиты растений',
 'повышение эффективности защиты',
 'увеличение урожайности сельскохозяйственных культур при использовании биостимулятора']

In [ ]:
import json

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import rule, Parser, or_, and_
from yargy.predicates import eq, type, gram, normalized
from yargy.pipelines import morph_pipeline
from random import seed, sample

from ipymarkup import show_span_ascii_markup as show_markup

def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )


class Match(object):
    def __init__(self, fact, spans):
        self.fact = fact
        self.spans = spans

ACTIONTitle = rule(
    "ACTION",
)
OBJECTTitle = rule(
    "OBJECT",
)
CONDITIONTitle = rule(
    "CONDITION",
)

TITLE = or_(
    ACTIONTitle,
    OBJECTTitle,
    CONDITIONTitle
)
    
DEBUG = or_(
    ACTION,
    OBJECT,
    CONDITION
)

class Extractor(object):
    def __init__(self):
        self.debug = Parser(DEBUG)
        # self.parser = Parser(TITLE)

    def __call__(self, line):
        matches = self.debug.findall(line)
        spans = [_.span for _ in matches]
        line = join_spans(line, spans)
        matches = self.parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        fact = None
        if matches:
            fact = matches[0].fact.item
        return Match(fact, spans)

extractor = Extractor()
# seed(10)

testText = "Изобретение относится к сельскому хозяйству и предназначено для повышения урожайности и защиты растений от комплекса фитопатогенов."
match = extractor(testText)
show_markup(testText, match.spans)
if match.fact:
    show_json(match.fact.as_json)


# AOC = rule(ACTION, OBJECT, CONDITION)
# parser = Parser(AOC)
# matches = list(parser.findall(text1))
# spans = [_.span for _ in matches]
# show_markup(text1, spans)

In [ ]:
text = '''
Изобретение относится к области высокотемпературной электрохимии, точнее к способам высокотемпературного электролиза и электрохимическим устройствам с твердым электролитом на основе двуокиси циркония. Изобретение может быть использовано для регенерации газовой атмосферы замкнутых объемов в системах жизнеобеспечения обитаемых космических кораблей.
Известно лабораторное устройство для высокотемпературного электрохимического разложения углекислого газа, воды или их смеси (Труды Института электрохимии УНЦ АН СССР, вып. №26, стр.102, 1978 г.). Устройство состоит из нагревателя, испарителя воды, конденсаторов паров воды, неразложившихся в электролизере и непосредственно батареи высокотемпературного электролизера с твердым электролитом на основе двуокиси циркония и газодиффузионными электродами. Если углекислый газ при нормальных условиях может непосредственно поступать в батарею, то воду необходимо предварительно нагреть и испарить. После этого газовая смесь углекислого газа и воды из испарителя поступают в катодную полость батареи электролизера. Под действием постоянного тока поступившие реагенты разлагаются на кислород и CO+H2, причем ионизированный кислород проходит через твердый электролит, выделяется в анодной полости на кислородном электроде и может использоваться по назначению. Остаток неразложившейся воды из газа конденсируется в жидкость в конденсаторе.
К недостаткам способа и устройства следует отнести недостаточный срок службы и ухудшение к концу срока удельных характеристик. Ухудшения начинаются после трехсот часов работы и достигают недопустимых величин после 1,5÷2 тысяч часов.
Наиболее близким по технической сущности (прототипом) следует считать аппарат для регенерации газовой атмосферы, в котором объединены в единой конструкции испаритель, конденсатор и нагреватель батареи высокотемпературного электролизера (авт. св-во СССР № 1840818).
Реагенты последовательно проходят испаритель, батарею и конденсаторы. В батарее реагенты разлагаются постоянным током, при этом кислород собирается в отдельной анодной полости, а остаток неразложившейся воды конденсируется в жидкость. Кислород и вода, пригодные для употребления, возвращаются для использования человеком по назначению.
К недостаткам известной конструкции аппарата и способа регенерации следует отнести относительно малый срок службы. После трехсот часов работы начинают ухудшаться удельные характеристики аппарата, которые достигают недопустимых величин после 1,5÷2 тысяч часов.
Целью настоящего изобретения является способ регенерации газовой атмосферы и аппарат для его реализации, позволяющие увеличить срок службы и повысить удельные характеристики аппарата в конце срока службы.
Указанная цель достигается тем, что реагенты, подлежащие разложению, поочередно вводят то на вход аппарата, то на выход, причем смену направления движения реагентов осуществляют по достижении поляризационного напряжения на первом элементе по потоку газа до 50-70 мВ. Для этого аппарат имеет дополнительно реверсирующее (коммутирующее) устройство для смены направлений газовых потоков, расположенное в тепловой зоне аппарата, исключающей конденсацию паров воды. Электрический сигнал для переключения может поступать с элементов, снабженных зондами, которые вследствие переключений могут стать первыми по газовому потоку.
На чертеже изображена блок-схема аппарата для регенерации газовой атмосферы, состоящего из батарей элементов  1, 2, 3 с твердым электролитом, реверсирующего (коммутирующего) устройства (КУ) 4, испарителя (И) 5, конденсатора (К) 6, нагревателя 7 и электрических зондов  8.
Батареи цилиндрических элементов изготовлены из твердого электролита состава 0,91 ZrO2 + 0,09 Se2O3, платиновых газодиффузионных электродов, нанесенных методом вжигания паст и выходящих на противоположные торцы элементов. Последовательное электрическое и механическое соединение элементов и механическое соединение элементов осуществляли термокомпрессионной сваркой. В качестве реверсирующего (коммутирующего) устройства использовали специально изготовленный многоходовой кран с обечайкой из фторопласта и вкладышем из нержавейки для надежной работы при +150°С. Поляризационное напряжение замеряли цифровым вольтметром Щ 68000 между газотокоподводом и зондом 8. По достижении величины напряжения значения уставки вольтметр выдает электрический сигнал исполняющему механизму для перевода многоходового крана на один шаг (ход). В исходном положении вода подается в испаритель 5 и через реверсирующее устройство 4 в последовательно соединенные по газу батареи 1, 2, 3, с выхода третьей батареи водород с остатком воды поступают в конденсатор-разделитель 6, из которого водород и вода идут по своим каналам. Путь газов в исходном I-м положении указан на чертеже стрелками. После перевода многоходового крана во II-м положение батареи также остаются последовательно соединенными, только газ входит во 2-ю батарею, проходит 3-ю и выходит из 1-й батареи. При этом напряжение уже измеряют на входном (первом) элементе, зондом 8 второй батареи. В III-м положении соответственно газ входит в 3-ю батарею, проходит через 1-ю и выходит через 2-ю. Затем цикл во времени повторяется. Каждая из батарей может представлять собой несколько соединенных параллельно по газу и току электролизных модулей. В каждой батарее разлагается 30÷33% входящих реагентов, а переключение осуществляют через 60-80 часов, т.е. полный цикл коммутации газовых потоков составляет 180-240 часов. Время и процент разложения авторами выбраны на основании исследований обратимости электродных процессов на элементах в атмосфере, богатой по воде. При увеличении поляризационного напряжения выше 70 мВ появляется необратимость электродных процессов. Осуществлять переключение при достижении менее 50 мВ нецелесообразно, т.к. увеличиваются энергозатраты на само переключение.
Благодаря использованию предлагаемого способа и конструкции аппарата срок его службы увеличивается не менее чем в 2÷3 раза. При этом удельные электрохимические характеристики аппарата через 3÷4 тысячи часов работы при 900°С (поляризационные потери) в 10-30 раз меньше. Все это позволяет существенно повысить срок службы и надежность работы аппарата для регенерации газовой атмосферы замкнутых объемов космических кораблей, подводных лодок и т.п.
'''

ТЕСТЫ

In [ ]:
from ipymarkup import show_span_ascii_markup as show_markup

import json
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp
from yargy import rule, Parser, or_, and_
from yargy.predicates import eq, type, gram, normalized
from yargy.pipelines import morph_pipeline

# Создание правил для "любого" слова
WORD = or_(
    gram('NOUN'),
    gram('ADJF'),
    gram('VERB')
)

Structure = fact(
    'Structure',
    ['A', 'O', 'C']
)

# Создание правил для сущности объект
OBJ1 = rule(
    and_(
      gram('NOUN'),
      gram('gent')  
    ) 
).interpretation(Structure.O.const("O1"))

OBJ2 = rule(
    OBJ1.optional(),
    gram('ADJF').optional(),
    OBJ1,
    WORD
).interpretation(Structure.O.const("O2"))
OBJ3 = rule(
    or_(
        rule(
            or_(OBJ1,OBJ2).repeatable(),
            rule(eq('и')),
            or_(OBJ1,OBJ2).repeatable(),
        ),
        rule(
            or_(OBJ1,OBJ2).repeatable(),
            rule(eq('и')),
            or_(OBJ1,OBJ2).repeatable(),
        )
    )
).interpretation(Structure.O.const("O3"))

# Создание правил для сущности условие
CON1 = rule(
    and_(
        WORD,
        or_(
          gram('loct'),
          gram('ablt')
        )
    )
).interpretation(Structure.C.const("C1"))
CON2 = rule(
    CON1,
    rule(
        and_(
        WORD,
        gram('gent')
      )
    ).repeatable()
).interpretation(Structure.C.const("C2"))
CON3 = rule(
    or_(
        rule(
            CON1,
            rule(eq('и')),
            and_(
                WORD,
                gram('nomn')
            )
        ),
        rule(
            CON2,
            rule(eq('и')),
            and_(
                WORD,
                gram('nomn')
            )
        )
    )
).interpretation(Structure.C.const("C3"))



# Правила струстуры AOC
ACTION = rule(morph_pipeline(actionVerbs)).interpretation(Structure.A.const("A"))
OBJECT = rule(or_(
    OBJ1,
    OBJ2,
    OBJ3
)).interpretation(Structure.O.const("OBJECT"))
CONDITION = rule(or_(
    CON1,
    CON2,
    CON3
).optional()).interpretation(Structure.C.const("CONDITION"))

def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )


AOC = rule(ACTION, OBJECT, CONDITION)

parser = Parser(OBJECT)

testText = "Изобретение относится к сельскому хозяйству и предназначено для повышения урожайности и защиты растений от комплекса фитопатогенов."

matches = parser.findall(text1)
spans = [_.span for _ in matches]
testText = join_spans(text1, spans)
print(testText)
print(spans)
print(matches)

print(matches[1].rule)
print(matches[1].fact)
text

к сельскому и повышения урожайности и защиты растений комплекса фитопатогенов защиты растений фитопатогенов и стрессов повышения продуктивности сельскохозяйственных культур при использовании препарата Альбит А К В Т и др альбит повышения урожая и защиты растений рекомендации применения М Агрорус с продукта метаболизма бактерий поли гидроксимасляной кислоты наработки препарата и требует оборудования повышения эффективности защиты фитопатогенов увеличения урожайности сельскохозяйственных культур при использовании биостимулятора вермикомпоста И А Брощак И С и карантин растений с
[[22, 33), [44, 45), [64, 103), [107, 130), [148, 163), [167, 191), [201, 246), [247, 272), [273, 289), [302, 303), [304, 305), [315, 316), [317, 318), [320, 324), [338, 344), [349, 383), [392, 404), [417, 427), [431, 432), [435, 442), [454, 455), [480, 509), [512, 516), [522, 546), [567, 586), [605, 614), [630, 642), [660, 690), [694, 707), [708, 751), [752, 777), [778, 792), [803, 816), [839, 840), [841, 842), [

TypeError: ignored

In [ ]:
pip install -U yargy

In [ ]:
from ipymarkup import show_span_ascii_markup as show_markup

from yargy import Parser, rule, and_
from yargy.interpretation import fact
from yargy.predicates import gram, normalized, caseless, type

AOC = fact('AOC', ['Action', 'Object', 'Condition'])

improve_verbs =  [
    'уменьшение',
    'повышение',
    'увеличение',
    'улучшение',
    'разделение',
    'расширение',
    'обеспечение',
    'компенсация',
    'выполнение',
    'облучение',
    'реализация',
    'предотвращение',
    'снижение',
    'исключение',
    'ускорение',
    'создание',
    'препятствие',
    'корректировать',
    'стабилизация',
    'формирование',
    'устранение',
    'посадку',
    'присыпать',
    'техническая задача',
    'оборачивание'
]

improve_verbs_rule = or_(*[normalized(verb) for verb in improve_verbs])

noun = gram('NOUN')
adjf = gram('ADJF')
adjs = gram('ADJS')
prep = gram('PREP')

sim_conj = caseless('или') , caseless('и')

Obj = fact('Obj', ['adjs','adjf', 'noun', 'prep'])

Obj_rule = rule(
    adjs.interpretation(Obj.adjs).optional().repeatable(),
    adjf.interpretation(Obj.adjf).optional().repeatable(),
    noun.interpretation(Obj.noun).optional(),
    prep.interpretation(Obj.prep).optional(),
)

Con = fact('Con', ['con', 'word', 'word_list'])

Con_rule = rule(
    rule(normalized('если').interpretation(Con.con)),
    rule(Obj_rule.interpretation(Con.word)).repeatable().interpretation(Con.word_list)
        .optional(True),
)

S = rule(
    improve_verbs_rule.interpretation(AOC.Action),
    Obj_rule.interpretation(AOC.Object),
    Con_rule.interpretation(AOC.Condition).optional().repeatable(),
).interpretation(AOC)

parser = Parser(S)

matches = list(parser.findall(text1))
spans = [_.span for _ in matches]
show_markup(text1, spans)